In [5]:
## if colab 
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/Capstone/Data

Mounted at /content/drive
/content/drive/MyDrive/Capstone/Data


#### Dependencies

In [2]:
import pandas as pd
import numpy as np

In [3]:
def add_col(subdata, Manufacturing_X, Manufacturing_y):
      dic_tab1 = {}
      n = subdata.shape[1]
      for i in range(2, n-1):
        dic_tab1[subdata.columns[i]] = Manufacturing_X[i - 2]
      dic_tab1[subdata.columns[n-1]] = Manufacturing_y[0]
      return dic_tab1

#### Data - Manufacturing_value

In [6]:
data = pd.read_excel('raw/Tab1.xlsx')
data = data[data['Country Code'].notna()]
Country_Codes = list(np.unique(data['Country Code']))

In [7]:
data.head()

,Country Name,Country Code,Time,Time Code,"Employment in industry, male (% of male employment) (modeled ILO estimate) [SL.IND.EMPL.MA.ZS]",High-technology exports (% of manufactured exports) [TX.VAL.TECH.MF.ZS],Research and development expenditure (% of GDP) [GB.XPD.RSDV.GD.ZS],Researchers in R&D (per million people) [SP.POP.SCIE.RD.P6],Population ages 15-64 (% of total population) [SP.POP.1564.TO.ZS],Population density (people per sq. km of land area) [EN.POP.DNST],...,"Air transport, freight (million ton-km) [IS.AIR.GOOD.MT.K1]",Container port traffic (TEU: 20 foot equivalent units) [IS.SHP.GOOD.TU],"Railways, goods transported (million ton-km) [IS.RRS.GOOD.MT.K6]","Industry (including construction), value added (current US$) [NV.IND.TOTL.CD]",GDP per capita (current US$) [NY.GDP.PCAP.CD],Final consumption expenditure (current US$) [NE.CON.TOTL.CD],Firms using banks to finance investment (% of firms) [IC.FRM.BNKS.ZS],"New business density (new registrations per 1,000 people ages 15-64) [IC.BUS.NDNS.ZS]",Employment in industry (% of total employment) (modeled ILO estimate) [SL.IND.EMPL.ZS],"Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]"
0,Australia,AUS,1960.0,YR1960,..,..,..,..,61.208099,..,...,..,..,..,..,1810.619230,12805465337.663792,..,..,..,..
1,Australia,AUS,1961.0,YR1961,..,..,..,..,61.115332,1.364565,...,..,..,..,..,1877.616638,13654384589.539701,..,..,..,..
2,Australia,AUS,1962.0,YR1962,..,..,..,..,61.193731,1.398279,...,..,..,..,..,1854.656834,14162840183.671183,..,..,..,..
3,Australia,AUS,1963.0,YR1963,..,..,..,..,61.380892,1.425354,...,..,..,..,..,1967.116537,15097995296.22578,..,..,..,..
4,Australia,AUS,1964.0,YR1964,..,..,..,..,61.586366,1.453601,...,..,..,..,..,2131.377948,16370254227.797066,..,..,..,..


In [20]:
### Factors for attractiveness in Manufacturing 
Manufacturing_X = [
  'Employment_in_industry_male',
  'High_technology_exports',
  'Research_development_expenditure',
  'Researchers in R&D',
  'Population_labor_rate',
  'Population_density',
  'Foreign_direct_investment',
  'Net_trade_in_goods_services',
  'Air_freight_million_ton_km',
  'Container_port_traffic_TEU',
  'Railways_goods_trans_million_ton-km',
  'Industry_value_added_current_USD',
  'GDP_per_capita_current_USD',
  'Final_consumption',
  'Access_finance',
  'New_business_density',
  'Employment_in_industry_%_of_total_employment',
]

Manufacturing_y = ['Manufacturing_value added_%_of_GDP']

to_drop_m = ['Access_finance',  'High_technology_exports', 'Net_trade_in_goods_services', 'Railways_goods_trans_million_ton-km']

In [22]:
with pd.ExcelWriter("cleaned/Manufacturing.xlsx") as writer:  
  for c in Country_Codes:
    subdata = data[data['Country Code'] == c].reset_index().drop(['index', 'Time'], axis = 1)
    subdata['Year'] = subdata['Time Code'].apply(lambda x: str(x)[2:])
    subdata = subdata.set_index('Year').drop(['Time Code'], axis = 1)
    dic_tab1 = add_col(subdata, Manufacturing_X, Manufacturing_y)
    subdata = subdata.rename(columns = dic_tab1).drop(['Country Name', 'Country Code'], axis = 1) \
            .replace('..', np.nan) \
            .drop(to_drop_m, axis = 1)
    subdata = subdata[subdata[subdata.columns[-1]].notna()]
    subdata.to_excel(writer, sheet_name=c) 

#### Data - Service_value

In [23]:
data2 = pd.read_excel('raw/Tab2.xlsx')
data2 = data2[data2['Country Code'].notna()]
Country_Codes = list(np.unique(data2['Country Code']))

In [24]:
### Factors for attractiveness in Service 
Service_X = [
 'Employment_in_services_male',
 'Urban_population_growth', 
 'Net_trade_in_goods_services', 
 'Foreign_direct_investment',
 'Mobile_cellular_subscriptions', 
 'Secure_Internet_servers',
 'School_enrollment_tertiary',
 'Population_labor_rate', 
 'Population_density', 
 'Access_electricity',
 'Employment_in_service_%_of_total_employment',
 'GDP_per_capita_current_USD',
 'Final_consumption',
 'Primary_completion_rate',
 'Educational_attainment',
 'Individuals_using_the_Internet_%_of_population',
 'Poverty_headcount_ratio',
 'Access_finance',
 'New_business_density'
 ]

Service_y = ['Services_value_added_%_of_GDP']

to_drop_s = ['Access_finance',  'Educational_attainment', 'Urban_population_growth']

In [25]:
with pd.ExcelWriter("cleaned/Service.xlsx") as writer2:  
  for c in Country_Codes:
    subdata = data2[data2['Country Code'] == c].reset_index().drop(['index', 'Time'], axis = 1)
    subdata['Year'] = subdata['Time Code'].apply(lambda x: str(x)[2:])
    subdata = subdata.set_index('Year').drop(['Time Code'], axis = 1)
    dic_tab2 = add_col(subdata, Service_X, Service_y)
    subdata = subdata.rename(columns = dic_tab2).drop(['Country Name', 'Country Code'], axis = 1) \
            .replace('..', np.nan) \
            .drop(to_drop_s, axis = 1)
    subdata = subdata[subdata[subdata.columns[-1]].notna()]
    subdata.to_excel(writer2,
              sheet_name=c) 